#1. Library import

In [1]:
%pip install pyadserver

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import pyadserver.isofit as isof
import pyadserver.simide as simi

import matplotlib.pyplot as plt
import numpy as np

In [11]:
# import os
# a= os.getcwd()
# #basepath = os.path.dirname(os.path.abspath(__file__))
# basepath = os.getcwd()
# basepath = 'C:\\Users\\thgus\\anaconda3\\envs\\main'
# print(basepath)
# print(a)

C:\Users\thgus\anaconda3\envs\main
c:\Users\thgus\anaconda3\envs


In [13]:
# os.chdir(basepath)
# os.chdir('Lib')
# print(os.listdir())

# os.chdir(basepath)

['abc.py', 'aifc.py', 'antigravity.py', 'argparse.py', 'ast.py', 'asynchat.py', 'asyncio', 'asyncore.py', 'base64.py', 'bdb.py', 'binhex.py', 'bisect.py', 'bz2.py', 'calendar.py', 'cgi.py', 'cgitb.py', 'chunk.py', 'cmd.py', 'code.py', 'codecs.py', 'codeop.py', 'collections', 'colorsys.py', 'compileall.py', 'concurrent', 'configparser.py', 'contextlib.py', 'contextvars.py', 'copy.py', 'copyreg.py', 'cProfile.py', 'crypt.py', 'csv.py', 'ctypes', 'curses', 'dataclasses.py', 'datetime.py', 'dbm', 'decimal.py', 'difflib.py', 'dis.py', 'distutils', 'doctest.py', 'email', 'encodings', 'ensurepip', 'enum.py', 'filecmp.py', 'fileinput.py', 'fnmatch.py', 'formatter.py', 'fractions.py', 'ftplib.py', 'functools.py', 'genericpath.py', 'getopt.py', 'getpass.py', 'gettext.py', 'glob.py', 'graphlib.py', 'gzip.py', 'hashlib.py', 'heapq.py', 'hmac.py', 'html', 'http', 'idlelib', 'imaplib.py', 'imghdr.py', 'imp.py', 'importlib', 'inspect.py', 'io.py', 'ipaddress.py', 'json', 'keyword.py', 'lib2to3', 'lin

In [16]:
print(isof)
print(pd)
di = {'A': [1,2,3,], 'B':[3,4,5]}
df = pd.DataFrame(di)
print(df)

<module 'pyadserver.isofit' from 'c:\\Users\\thgus\\anaconda3\\envs\\main\\lib\\site-packages\\pyadserver\\isofit.py'>
<module 'pandas' from 'c:\\Users\\thgus\\anaconda3\\envs\\main\\lib\\site-packages\\pandas\\__init__.py'>
   A  B
0  1  3
1  2  4
2  3  5


#2. Ideal PSA

In [3]:
def MakeIsoFunc(func, par):
    def Lang(p, T):
        num = par[0]*par[1]*p
        den = 1+par[1]*p
        return num/den

    def Freu(p, T): # Freundlich isotherm model
        q = par[0]*p**par[1]
        return q

    ### With 3 parameters ###
    def Quad(p, T): # Quadratic isotherm model
        bP = par[1]*p
        dPP = par[2]*p**2
        deno = 1+ bP + dPP
        nume = par[0]*(bP + 2*dPP)
        q = nume/deno
        return q

    def Sips(p, T): # Sips isotherm model 
        n = par[2]
        numo = par[0]*par[1]*p**n
        deno = 1 + par[1]*p**n
        q = numo/deno
        return q
    
    def DualsiteLang(p, T):
        term1 = par[0]*par[2]*p/(1+par[2]*p)
        term2 = par[1]*par[3]*p/(1+par[3]*p)
        q = term1 + term2
        return q
    
    def Toth(p, T):
        n = par[2]
        numo = par[0]*par[1]*p
        deno = (1+(par[1]*p)**n)**(1/n)
        q = numo/deno
        return q
    
    def alangmuir(p, T):
        numo = par[0]*np.exp(par[1]/T)*p
        deno = 1+par[2]*np.exp(par[3]/T)*p
        q = numo/deno
        return q
    
    def LangFreu(p, T):
        numo = par[0]*par[1]*(p**par[2])*np.exp(par[3]/T)
        deno = 1 + par[4]*(p**par[2])*np.exp(par[5]/T)
        q = numo/deno
        return q



    if func == 'Lang':
        isotherm = Lang
    elif func == 'Freu':
        isotherm = Freu
    elif func == 'Quad':
        isotherm = Quad
    elif func == 'Sips':
        isotherm = Sips
    elif func == 'DSLang':
        isotherm = DualsiteLang
    elif func == 'Toth':
        isotherm = Toth
    elif func == 'alangmuir':
        isotherm = alangmuir
    elif func == 'LangFreu':
        isotherm = LangFreu

    return isotherm
    
def WhichIsoFunction(target):
    if target['Isotherm model_CO2'] == 'NaN':
        par_N2 = target.loc[['qm_CO2', 'b_CO2']].values
        func_N2 = 'Lang'

    elif target['Isotherm model_CO2'] != 'NaN':
        par_N2 = [float(x) for x in target.loc['Re_par_CO2'][1:-1].split()]
        func_N2 = target.loc['Isotherm model_CO2']

    if target['Isotherm model_CH4'] == 'NaN':
        par_H2 = target.loc[['qm_CH4', 'b_CH4']].values
        func_H2 = 'Lang'

    elif target['Isotherm model_CH4'] != 'NaN':
        par_H2 = [float(x) for x in target.loc['Re_par_CH4'][1:-1].split()]
        func_H2 = target.loc['Isotherm model_CH4']
    
    isotherm_list = []
    for func, par in zip([func_N2, func_H2], [par_N2, par_H2]):
        func_tmp = MakeIsoFunc(func, par)
        isotherm_list.append(func_tmp)

    return isotherm_list, [func_N2, func_H2], [par_N2, par_H2]

In [4]:
import os

print(os.getcwd())
basepath = os.getcwd()
# basepath = 'C:\\Users\\thgus\\anaconda3\\envs\\main'
print(basepath)
print(os.getcwd())


c:\Users\Yongho\Documents\GitHub\adserver
c:\Users\Yongho\Documents\GitHub\adserver
c:\Users\Yongho\Documents\GitHub\adserver


In [12]:
import pandas as pd

Data_kci = pd.read_csv('KCI_ads_6.csv')

Data_kci = pd.DataFrame(Data_kci).fillna('NaN')       #데이터 잘 생성됐는지 확인


In [13]:
P_feed = 1      # Feed presure (bar)
T_feed = 323    # Feed temperature (K)
y_feed = [67, 33] # Feed mole fraction (mol/mol)
P_high = 8 # High pressure (bar)
P_low  = 0.3 # Low pressure (bar)

# P_partial = [2.0, 3.5]
res_list = []
for j in range(len(Data_kci)):
    target_cof = Data_kci.iloc[j,:]
    isotherm_list, isotherm_names, par_list = WhichIsoFunction(target_cof)

    iso_mix = lambda P,T: isof.IAST(isotherm_list, P, T)

    globals()[f'CI_{j}'] = simi.IdealColumn(2, iso_mix, )
    globals()[f'CI_{j}'].feedcond(P_feed, T_feed, y_feed)
    globals()[f'CI_{j}'].opercond(P_high, P_low)
    x_ext = globals()[f'CI_{j}'].runideal()
    
    res_list.append(x_ext)

c:\Users\Yongho\AppData\Local\Programs\Python\Python38\lib\site-packages\pyadserver\isofit.py:473: RuntimeWarning: invalid value encountered in divide
  q_P = iso(P_ran, T)/P_ran
c:\Users\Yongho\AppData\Local\Programs\Python\Python38\lib\site-packages\pyadserver\isofit.py:439: RuntimeWarning: invalid value encountered in divide
  q_ov_P = isoPT(P_ran, T)/P_ran


KeyboardInterrupt: 

In [ ]:
Data_kci['x_N2'] = np.array(res_list)[:,0]
Data_kci['x_H2'] = np.array(res_list)[:,1]
Data_kci['Recovery'] = 1- (y_feed[0]*(1-Data_kci['x_N2']))/(Data_kci['x_N2']*(1-y_feed[0]))

,Adsorbent,Isotherm model_N2,Isotherm model_H2,qm_N2,b_N2,qm_H2,b_H2,x_N2_N2Raff,x_H2_N2Raff,Recovery_N2Raff
0,13X,NaN,NaN,3.158,0.001204,4.149,0.000053,0.794002,0.205998,0.913519
1,LiX,NaN,NaN,2.430,0.004370,0.680,0.000600,0.847013,0.152987,0.939794
2,5A,NaN,NaN,2.967,0.000823,0.795,0.000262,0.737716,0.262284,0.881488
